In [ ]:
!pip install qiskit;

     |████████████████████████████████| 8.5MB 5.3MB/s 
     |████████████████████████████████| 17.6MB 253kB/s 
     |████████████████████████████████| 204kB 47.6MB/s 
     |████████████████████████████████| 204kB 54.8MB/s 
     |████████████████████████████████| 2.1MB 48.1MB/s 
     |████████████████████████████████| 1.3MB 47.1MB/s 
     |████████████████████████████████| 5.9MB 51.0MB/s 
     |████████████████████████████████| 51kB 5.8MB/s 
     |████████████████████████████████| 194kB 45.0MB/s 
     |████████████████████████████████| 81kB 9.1MB/s 
     |████████████████████████████████| 583kB 34.4MB/s 
     |████████████████████████████████| 102kB 9.5MB/s 
     |████████████████████████████████| 3.2MB 8.8MB/s 
     |████████████████████████████████| 5.5MB 26.8MB/s 
  Created wheel for qiskit: filename=qiskit-0.23.5-cp36-none-any.whl size=2882 sha256=f2ca72f7616fedd93b245eff14136c9680abe650f678cb2db70dc6b13f086aa7
  Stored in directory: /root/.cache/pip/wheels/0a/a7/8f/2720a308d16a991c

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.autograd import Function
from torchvision import datasets, transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

import qiskit
from qiskit.visualization import *

In [ ]:
# Defines the quantum circuit so we can use it (since hybrids have quantum)
class QuantumCircuit:
  #This is the initialization
  def __init__(self, n_qubits, backend, shots):
    #Define how many lanes we want
    self._circuit = qiskit.QuantumCircuit(n_qubits)

    #Just a list of 0 to the number of qubits... Just useful so we can just define the circuit (with all it's little parts super quickly)
    all_qubits = [i for i in range(n_qubits)]
    #Kind of like a placeholder variable so we can fill it in later
    self.theta = qiskit.circuit.Parameter('theta') #Change when multi

    #Shove in the Hardav gate, a barrier (visual), and a rotation about the y plane of theta degrees
    self._circuit.h(all_qubits)
    self._circuit.barrier()
    self._circuit.ry(self.theta, all_qubits) #Change when multi

    self._circuit.measure_all()

    #save these varaibles for later so we don't have to call them again during the forwarding
    self.backend = backend
    self.shots = shots
  
  #forwarding through the quantum circuit
  def run(self, thetas):
    #prep the execution. Link to circuit, Define backend and number of shots... And then fill in the placeholder variable (theta) with the thing we pass through when we forward
    job = qiskit.execute(self._circuit,
                         self.backend,
                         shots = self.shots,
                         parameter_binds = [{self.theta: theta} for theta in thetas]) # Might have to change this when multi layer
    
    #execution
    result = job.result().get_counts(self._circuit)

    #Counts = number of occurances (for each state)
    counts = np.array(list(result.values()))
    
    #States=  States of the occuraces (like 001, 101, 110 etc)
    states = np.array(list(result.keys())).astype(float)

    #This just brings it down to the percentage that this particular state occurs
    prob = counts / self.shots

    #AKA what the average of this circuit does
    expect = np.sum(states * prob)
    #Returns as an array
    return np.array([expect])

In [ ]:
sim = qiskit.Aer.get_backend('qasm_simulator')

test_circuit = QuantumCircuit(1, sim, 1000)

In [ ]:
test_circuit.run([3/2])

test_circuit._circuit.draw()

┌───┐ ░ ┌───────────┐ ░ ┌─┐
   q_0: ┤ H ├─░─┤ RY(theta) ├─░─┤M├
        └───┘ ░ └───────────┘ ░ └╥┘
meas: 1/═════════════════════════╩═
                                 0

In [ ]:
#This class defines what our hybrid layer does. It allows it to go forward, and also backprop
class HybridFunction(Function):

  @staticmethod
  def forward(ctx, input, quantum_circuit, shift):
    ctx.shift = shift
    ctx.quantum_circuit = quantum_circuit

    #Aka we run the input into our circuit
    expectation_z = ctx.quantum_circuit.run(input[0].tolist()) #Might need to change this when have multi-layer
    #Shoves to pytorch tesnor
    result = torch.tensor([expectation_z])
    #Save the input and result for backpropagation
    ctx.save_for_backward(input,result)
    #return output
    return result
  
  @staticmethod
  def backward(ctx, grad_output):
    #Regrabing the input and the output
    input, expectation_z = ctx.saved_tensors
    input_list = np.array(input.tolist())

    #AKA if we take our inputs and just add/subtract a tiny amount
    shift_right = input_list + np.ones(input_list.shape) * ctx.shift
    shift_left = input_list - np.ones(input_list.shape) * ctx.shift

    #A list of all the gradients
    gradients = []

    #We're going to go through the inputs and then calculate the gradient for each one
    for i in range(len(input_list)):
      #So we take the shifted ones and just compute it 
      expectation_right = ctx.quantum_circuit.run(shift_right[i])
      expectation_left  = ctx.quantum_circuit.run(shift_left[i])
      #Gradient = appox the difference (division by 2*shift isn't necessary since it'll just be a scaled version (which can be counteracted by lr))
      gradient = torch.tensor([expectation_right]) - torch.tensor([expectation_left])
      #Append the gradient to the meta list
      gradients.append(gradient)
    
    #Turn it into np and transpose it
    gradients = np.array([gradients]).T
    #then return it for backprop
    print(gradients)
    print(torch.tensor([gradients]).float() * grad_output.float(), None, None)
    return torch.tensor([gradients]).float() * grad_output.float(), None, None

In [ ]:
#This defines the acutal thing that we're shoving into the NN
class Hybrid(nn.Module):
  #initialization
  def __init__(self, backend, shots, shift):
    super(Hybrid, self).__init__()
    #Define the real quantum circuit that we'll be using for our thing
    self.quantum_circuit = QuantumCircuit(1, backend, shots)
    #Save this guy for alter
    self.shift = shift
  
  #When forwarding
  def forward(self, input):
    return HybridFunction.apply(input, self.quantum_circuit, self.shift)

In [ ]:
#Now we just define the data stuff

n_samples = 1000

X_train = datasets.MNIST(root='./data', train=True, download=True,
                         transform=transforms.Compose([transforms.ToTensor()]))

# Leaving only labels 0 and 1 
idx = np.append(np.where(X_train.targets == 0)[0][:n_samples], 
                np.where(X_train.targets == 1)[0][:n_samples])

X_train.data = X_train.data[idx]
X_train.targets = X_train.targets[idx]

train_loader = torch.utils.data.DataLoader(X_train, batch_size=1, shuffle=True)

In [ ]:
n_samples = 1000

X_test = datasets.MNIST(root='./data', train=False, download=True,
                        transform=transforms.Compose([transforms.ToTensor()]))

idx = np.append(np.where(X_test.targets == 0)[0][:n_samples], 
                np.where(X_test.targets == 1)[0][:n_samples])

X_test.data = X_test.data[idx]
X_test.targets = X_test.targets[idx]

test_loader = torch.utils.data.DataLoader(X_test, batch_size=1, shuffle=True)

In [ ]:
#The actual one
class Net(nn.Module):
  #Initialization
  def __init__(self):
    super(Net, self).__init__()
    #Defining all the classical layers
    self.classical = nn.Sequential(
        nn.Conv2d(1,6,5),
        nn.ReLU(),
        nn.MaxPool2d(2),
        nn.Conv2d(6,16,5),
        nn.ReLU(),
        nn.MaxPool2d(2),
        nn.Dropout()
    )
    #Defining the middel part
    self.middel = nn.Sequential(
        nn.Linear(256,64),
        nn.ReLU(),
        nn.Linear(64,1))
    #Definign the quantum part
    self.quantum = Hybrid(qiskit.Aer.get_backend('qasm_simulator'), 100, np.pi / 2)
  
  #For whenever we pass stuff through it
  def forward(self,x):
    #Pass it through each one
    x = self.classical(x)
    x = x.view(1, -1)
    x = self.middel(x)
    theta = x
    x = self.quantum(x)
    return torch.cat((x,1-x), -1), theta

In [ ]:
#Define the model, optimzier and loss function
model = Net()
optimizer = optim.Adam(model.parameters(), lr = 0.001)
loss_func = nn.NLLLoss()

#Define the epochs and the list (where we plot) the losses
epochs = 5
loss_list = []
thetas = []

In [ ]:
#Training mode
model.train()
for epoch in range(epochs):
  #So we can shove it on the loss graph
  total_loss = []
  #Go through the dataloader
  for image, label in train_loader:
    #Set the gradient back to 0
    optimizer.zero_grad()
    #Shove the image through the model
    prediction,theta = model(image)
    #Get the loss
    loss = loss_func(prediction, label)
    #Compute the gradients
    loss.backward()
    #Update parameters
    optimizer.step()

    #Save theta
    thetas.append(theta)
    #Add this loss to the total
    total_loss.append(loss.item())
  
  #Shove total loss of epoch to the total loss of the model
  loss_list.append(sum(total_loss) / len(total_loss))
  #Print some readout so we know it's working
  print(loss_list[-1])

[[1.]]
tensor([[[-1.]]]) None None
[[0.99]]
tensor([[[-0.9900]]]) None None
[[0.99]]
tensor([[[-0.9900]]]) None None
[[0.97]]
tensor([[[0.9700]]]) None None
[[0.97]]
tensor([[[-0.9700]]]) None None
[[0.98]]
tensor([[[0.9800]]]) None None
[[0.98]]
tensor([[[-0.9800]]]) None None
[[0.93]]
tensor([[[0.9300]]]) None None
[[0.96]]
tensor([[[-0.9600]]]) None None
[[0.96]]
tensor([[[0.9600]]]) None None
[[0.93]]
tensor([[[-0.9300]]]) None None
[[0.96]]
tensor([[[0.9600]]]) None None
[[0.96]]
tensor([[[0.9600]]]) None None
[[0.92]]
tensor([[[-0.9200]]]) None None
[[0.87]]
tensor([[[-0.8700]]]) None None
[[0.98]]
tensor([[[0.9800]]]) None None
[[0.96]]
tensor([[[0.9600]]]) None None
[[0.98]]
tensor([[[0.9800]]]) None None
[[0.98]]
tensor([[[0.9800]]]) None None
[[0.89]]
tensor([[[-0.8900]]]) None None
[[0.77]]
tensor([[[-0.7700]]]) None None
[[0.95]]
tensor([[[0.9500]]]) None None
[[0.76]]
tensor([[[-0.7600]]]) None None
[[0.97]]
tensor([[[0.9700]]]) None None
[[0.99]]
tensor([[[0.9900]]]) None

KeyboardInterrupt: ignored

In [ ]:
plt.plot(thetas)

In [ ]:
thetas[-1]

In [ ]:
sim = qiskit.Aer.get_backend('qasm_simulator')

test_circuit = QuantumCircuit(1, sim, 1000)

In [ ]:
test_circuit.run([-1.9])

# test_circuit._circuit.draw()

In [ ]:
#Evaluation mode
model.eval()
#So it doesn't compute gradients
with torch.no_grad():
  correct = 0
  #Go through the test loader
  for image, label in test_loader:
    #Get prediction
    prediction,theta = model(image)
    #Get the index of the prediction (thus the true prediction of the image)
    true_pred = prediction.argmax(dim=1, keepdim = True)
    #If they're correct add em up
    correct += true_pred.eq(label.view_as(true_pred)).sum().item()
  
  print(correct / len(test_loader) * 100)